***
### Import of required libraries
***

In [ ]:
from traffic.data import opensky

from traffic.core import Traffic
import pandas as pd

***
### Import of flightplan data
***

In [ ]:
flightplan_path = "/mnt/beegfs/store/Projects_CRM/RAD_paper/flightplans/flightplans_complete.parquet"
flightplans = pd.read_parquet(flightplan_path)

***
### Download trajectory data per airport pair and direction
***

##### Function to remove invalid trajectories

In [ ]:
def remove_invalid(flight):
    if (
        (len(flight.data) > 600)
        and (flight.data.altitude.max() > 10000)
        and (flight.cumulative_distance().data.cumdist.max() >= 10)
    ):
        return flight

##### London airports to consider

In [ ]:
london_airports = [
    "EGLL",  # Heathrow
    "EGLC",  # London City
    "EGKK",  # Gatwick
]

##### Fetching of data

In [ ]:
# Path where to store the trajectory data
base_path = "/store/Projects_CRM/RAD_paper/trajectories/new/"

# Iterate over london airports
for airport in london_airports:

    # Fetch Zurich to London
    print(f"processing Zurich to London {airport}")
    callsigns = flightplans.query(
        "origin == 'LSZH' and destination == @airport"
    ).callsign.unique()
    t = opensky.history(
        start="2023-05-01 00:00:00",
        stop="2024-05-02 00:00:00",
        callsign=callsigns,
    )
    if t is not None:
        t = t.clean_invalid().assign_id().eval(max_workers=20, desc="preprocessing")
        t = t.iterate_lazy().pipe(remove_invalid).eval(max_workers=20, desc="removing invalid")
        t.to_parquet(f"{base_path}zurich_to_london_{airport}.parquet")

    # Fetch London to Zurich
    print(f"processing London {airport} to Zurich")
    callsigns = flightplans.query(
        "origin == @airport and destination == 'LSZH'"
    ).callsign.unique()

    t = opensky.history(
        start="2023-05-01 00:00:00",
        stop="2024-05-02 00:00:00",
        callsign=callsigns,
    )
    if t is not None:
        t = t.clean_invalid().assign_id().eval(max_workers=20, desc="preprocessing")
        t = t.iterate_lazy().pipe(remove_invalid).eval(max_workers=20, desc="removing invalid")
        t.to_parquet(f"{base_path}london_to_zurich_{airport}.parquet")